In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import csv
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import pandas as pd
import nltk
nltk.download('punkt_tab')

ModuleNotFoundError: No module named 'nltk'

In [3]:
import sys
!conda install --yes --prefix {sys.prefix} numpy matplotlib pandas nltk word2vec gensim


done
Solving environment: Out of memory allocating 18446744071562067968*4 bytes!


In [ ]:
def loadTwitterdata():
    '''# Load  "Twitter US Airline Sentiment". It can be found on kaggle 
    https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment
    and contains ~15000 tweets on airline-relevant topics from 2015. 
    '''

    def load_data(filename):
        rtn = []
        #open the file with the csv reader
        with open(filename, newline='') as csvfile:
            tweets = csv.reader(csvfile, delimiter=',', quotechar='"') 
            next(tweets)#skip the header   
            for row in tweets:
                rtn.append(row[10])
        return rtn
    tweets = load_data('Tweets.csv')
    #figure out how much data we have
    size = sum([i.__sizeof__() for i in tweets]) + tweets.__sizeof__()
    print('Number: ', len(tweets), '\t Size:', size/1e6,'MB','\t Bytes per tweet:', size/len(tweets))
    return(tweets)
    
tweets = loadTwitterdata()


In [ ]:
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html
# the word vectors, a 3M x 300 matrix, must be downloaded.
import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))


In [ ]:
# object-oriented everything, we must create an instance of the word2vec engine
wv = gensim.downloader.load('word2vec-google-news-300')

In [ ]:
# According to https://stackoverflow.com/questions/35596031/gensim-word2vec-find-number-of-words-in-vocabulary 
# wv acts like a dictionary. 
len(wv)  # gives me the number of items in vocabulary.

In [ ]:
wv["Chicago"]

In [ ]:
plt.imshow(wv["Chicago"].reshape((300,1)).T )

Meh, not the most brilliant visualization here

In [ ]:
plt.hist(wv["Chicago"], bins=30)

In [ ]:
# with a suggestion from google AI, start with a list of words, create a list of vectors, and
# convert both into a dataframe (which will handle the labels more nicely)
words = ['Chicago', "Boston", "Philadelphia", "lawyer", "attorney", "cook", "waiter", "thylacine", "wallaby"]
vectors = [wv[word] for word in words]
df = pd.DataFrame(vectors, index=words)


In [ ]:
df

In [ ]:
plt.imshow(df.to_numpy() , aspect="auto", interpolation="nearest") #, ylabels=words)
plt.yticks(range(len(words)), words)
plt.xlim([0,100])

In [ ]:
import seaborn as sns
sns.clustermap(d)

In [ ]:
sns.clustermap(df)

In [ ]:
# google code suggestion for "all-against-all distances pandas dataframe of vectors"
# clearly cdist does the n^2 / 2 distance calculations, and DataFrame just makes it pretty.
from scipy.spatial.distance import cdist

def pairwise_distances(df, metric='euclidean'):
    """Calculate pairwise distances for a DataFrame of vectors."""
    return pd.DataFrame(cdist(df.values, df.values, metric=metric), index=df.index, columns=df.index)


In [ ]:
d = pairwise_distances(df)
d

In [ ]:
plt.imshow(d , aspect="auto", interpolation="nearest")

plt.yticks(range(len(words)), words)
plt.xticks(range(len(words)), words, rotation = 45)
plt.colorbar()

In [ ]:
# Okay, this demo has wv.most_similar.. which takes a "positive" and a "negative" list of words
# https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.most_similar.html
print(wv.most_similar(positive=['mushroom'], topn=5))

In [ ]:
print(wv.most_similar(positive=['lawyers', "thylacine"], negative=["lawyer"], topn=5))


In [ ]:
print(wv.most_similar(positive=[wv["air"]/2 + wv["water"] / 2 ], topn=4))
print(wv.most_similar(positive=[wv["air"]/2 + wv["fire"] / 2 ], topn=4))
print(wv.most_similar(positive=[wv["air"]/2 + wv["earth"] / 2 ], topn=4))
print(wv.most_similar(positive=[wv["water"]/2 + wv["earth"] / 2 ], topn=4))
print(wv.most_similar(positive=[wv["fire"]/2 + wv["earth"] / 2 ], topn=4))
print(wv.most_similar(positive=[wv["water"]/2 + wv["fire"] / 2 ], topn=4))


In [ ]:
print(wv.most_similar(positive=['air', 'water'], topn=5))


In [ ]:
# Let's just average everything together.. (This isn't going to bite us later, is it?)
def string_to_vec(s):
    N=300
    n=0
    t = np.zeros(300)
    tok1 = nltk.word_tokenize(s)
    for token in tok1:
        try:
            c = wv[token]
            t = t+c
            n = n+1
        except KeyError:
            pass
    return(t / n)

In [ ]:
twittervectors = [string_to_vec(tweet) for tweet in tweets] 
twittervectors_small = [string_to_vec(tweet) for tweet in tweets[0:200]] 

In [ ]:
twitframe_small  = pd.DataFrame(twittervectors_small, index=tweets[0:200])


In [ ]:
sns.clustermap(twitframe)

In [ ]:
twit_d = pairwise_distances(twitframe_small)
sns.clustermap(twit_d)

In [ ]:
plt.hist(np.array(twit_d).reshape(-1), bins=100)

In [ ]:
print(wv.most_similar(positive=['disappointment'], topn=5))

In [ ]:
# Okay, Cookie, which one of these things is not like the others?
print(wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car']))

In [ ]:
print (wv.most_similar(positive=["Merced", "State"], negative=["City"]))

In [ ]:
print (wv.most_similar(positive=["Merced", "Chicago"], negative=["Illinois"]))

In [ ]:
print (wv.most_similar(positive=["Chicago", "State"], negative=["City"]))

In [ ]:
# https://radimrehurek.com/gensim/models/word2vec.html
from gensim.models import KeyedVectors

In [ ]:
vector = wv['computer']  # Get word2vec number for a word 
vector

In [ ]:
plt.hist(vector, bins=30)